Pirmame laboratoriniame darbe buvo pateikta duomenų bazė, užpildyta testiniais duomenimis, reikėjo atlikti 4 užklausas.

### Duomenų bazės pildymas

### Duomenų bazės schema
![](dbschema.png)
#### knyga
![](knyga.png)
#### autorius
![](autorius.png)
#### skaitytojas
![](skaitytojas.png)
#### egzempliorius
![](egzempliorius.png)

### Užduotys

1. **Visų egzempliorių, paimtų konkretaus skaitytojo, nurodyto jo numeriu, skaičius.**

In [ ]:
/* Atvejis, kai įvedame pasirinktą skaitytojo numerį į dvi vietas */
SELECT 1000 AS skaitytojas,
       COUNT(*) AS paimta
FROM Egzempliorius
WHERE Egzempliorius.Skaitytojas = 1000;

/* Atvejis, kai įvedame pasirinktą skaitytojo numerį į vieną vietą naudodami kintamojo priskyrimą (bet neišvengiame laikinųjų lentelių) */

DO $$
DECLARE n integer;
BEGIN SELECT 1000 INTO n;
DROP TABLE IF EXISTS tmp_table;
CREATE TEMPORARY TABLE tmp_table AS
SELECT n AS skaitytojas,
       COUNT(*) AS paimta
FROM Egzempliorius
WHERE Egzempliorius.Skaitytojas = 1000;
END $$;
SELECT *
FROM tmp_table;

![](dbvs_out11.png)

2. **Konkretaus skaitytojo, nurodyto jo AK, kiekvieno skaitomo egzemplioriaus numeris ir paėmimo bei grąžinimo datos.**

In [ ]:
SELECT Egzempliorius.Nr,
       TO_CHAR(Egzempliorius.Paimta, 'YYYY-MM-DD') AS Paimta,
       TO_CHAR(Egzempliorius.Grazinti, 'YYYY-MM-DD') AS Grazinti
FROM Egzempliorius,
     Skaitytojas
WHERE Egzempliorius.Skaitytojas = Skaitytojas.Nr
  AND Skaitytojas.AK='49903021111';

![](dbvs_out12.png)

3. **Kiekvienam ISBN pirmųjų simbolių ketvertui visų knygų ir visų jų egzempliorių skaičiai.**

In [ ]:
SELECT SUBSTRING(Egzempliorius.ISBN, 1, 4) AS ISBN_pradzia,
       COUNT(Egzempliorius.Nr) AS egzemplioriai,
       COUNT(Egzempliorius.Nr) AS knygos
FROM Egzempliorius,
     Knyga
WHERE Egzempliorius.ISBN = Knyga.ISBN
GROUP BY ISBN_pradzia;

![](dbvs_out13.png)

4. **Kiekvieneriems metams, kai buvo išleista bent viena knyga, pavadinimas ir ISBN tais metais išleistos knygos, kurios egzempliorių yra paimta daugiausiai.**

In [ ]:
/* Laikinoje lentelėje yra paimtų knygų skaičius kiekvienam ISBN taip pat nurodant su ISBN susietą pavadinimą ir metus;
Toliau sprendžiame klausimą, kaip SELECT frazėje su grupavimu prijungti su ISBN susietus stulpelius 'pavadinimas' ir 'metai' pagal https://stackoverflow.com/a/14841015/3044825 siūlomą atsakymą*/


WITH paimtos AS
  (SELECT knyga.ISBN,
          pavadinimas,
          metai,
          COUNT(paimta) AS paimtu_kiekis
   FROM egzempliorius
   JOIN knyga ON knyga.ISBN=egzempliorius.ISBN
   WHERE paimta IS NOT NULL
     AND metai IS NOT NULL
   GROUP BY knyga.ISBN)
SELECT p1.isbn,
       p1.pavadinimas,
       p1.metai,
       p1.paimtu_kiekis
FROM paimtos p1
LEFT JOIN paimtos p2 ON p1.metai=p2.metai
AND p1.paimtu_kiekis<p2.paimtu_kiekis
WHERE p2.paimtu_kiekis IS NULL;

![](dbvs_out14.png)

### Kitos problemos, susijusios su pirmu laboratoriniu

1. Pirmas uždavinys paskatino [pasidomėti](https://stackoverflow.com/questions/1490942/how-to-declare-a-variable-in-a-postgresql-query), kaip padaryti lentelę, kurioje rodo įvestą skaičių ir jo dažnumą lentelėje. Radau apėjimą, kuris naudoja DO komandą, veikiančią nuo Postgresql 9.0 versijos. Tačiau [SQL skriptas iš CodeSignal](https://app.codesignal.com/arcade/db/time-river-revisited/G9zoZjTzk6JpJGFbd), naudojamas priskyrimui, skiriasi. 

##### Rextester

##### Codesignal

2. Ketvirtame uždavinyje susidūriau su problema, kai neįmanoma vienu metu sugrupuoti lentelės pagal kažkurį stulpelį ir įterpti kitų stulpelių, nei tas, pagal kurį grupuojamos maksimalios reikšmės. 
Kitaip tariant, toks sprendimas netinka:
    


[Stackoverflow](https://stackoverflow.com/questions/3491329/group-by-with-maxdate) siūlo tokius sprendimo būdus:

##### Basic, multiple passes

In [ ]:
WITH Y AS
  (SELECT a, MAX(c) AS c
   FROM X
   GROUP BY a)
SELECT X.a, X.b, X.c
FROM X INNER JOIN Y ON (X.a = Y.a) AND (X.c=Y.c);

##### One pass, uses ranking

In [ ]:
With Y AS(
    SELECT a, b, c, 
    RANK() OVER (PARTITION BY a ORDER BY b DESC) d
    FROM X
    )
SELECT a, b, c
FROM Y
WHERE d = 1;

##### Clever, looks optimised but actually slow